## Lesson 7. Full Backup - STREAM Mode

### Theory: What is WAL?

Write-Ahead Log

Every change you make to the database - INSERT, UPDATE, DELETE -
is first written to WAL before being applied to the actual data files.
This ensures durability: even if the server crashes, PostgreSQL can replay WAL to recover.



### Theory: What is STREAM mode?

**STREAM mode** is a backup mode where WAL files are copied directly via the streaming replication protocol and stored inside the backup itself.

**Advantages of STREAM mode:**
- Does not require `archive_command` configuration
- Backup is completely self-contained
- Suitable for simple scenarios without continuous WAL archiving

**When to use:**
- When continuous WAL archiving is not configured
- For creating standalone backups
- For test and dev environments

**PostgreSQL requirements:**
- `wal_level = replica` (or higher `wal_level=logical`)
- `max_wal_senders >= 1` (for streaming connection)
- Replication permission in `pg_hba.conf`

### 1. Setup environment variables

In [1]:
import os

PG_BIN = '/usr/local/pgsql-17/bin'
os.environ['PG_CONFIG'] = f'{PG_BIN}/pg_config'
os.environ['PGPROBACKUPBIN'] = f'{PG_BIN}/pg_probackup'
os.environ['PGPROBACKUP_TMP_DIR'] = 'pg_probackup_demo'


### 2. Initialize pg_probackup2 environment

In [2]:
import testgres
from pg_probackup2.app import ProbackupApp
from pg_probackup2.init_helpers import Init
from pg_probackup2.storage.fs_backup import FSTestBackupDir


init_params = Init()
pg_node = testgres.NodeApp()

backup_dir = FSTestBackupDir(rel_path='rel_backup_dir', backup='backup_demo')
print(f"Backup catalog directory:\n{backup_dir.path}")

pb = ProbackupApp(
    pg_node=pg_node,
    pb_log_path=os.path.join(backup_dir.path, 'log'),
    backup_dir=backup_dir,
    probackup_path=init_params.probackup_path
)

Backup catalog directory:
/home/vshepard/PycharmProjects/backup_course/v7_full_backup_stream/pg_probackup_demo/rel_backup_dir/backup_demo


### 3. Create and configure PostgreSQL cluster for STREAM mode

To enable STREAM mode, PostgreSQL must be configured:

**postgresql.conf:**
```
wal_level = replica
max_wal_senders = 2
```

**pg_hba.conf:**
```
local replication all trust
host replication all 127.0.0.1/32 trust
```


> **⚠️ WARNING:** The `trust` authentication method allows connections without a password. This is **insecure** and should only be used for local testing/development. In production, use `scram-sha-256` or `md5`:
> ```
> host replication backup_user 127.0.0.1/32 scram-sha-256
> ```

a. Create PostgreSQL node with streaming replication settings
> initdb -D <data_directory>

In [3]:
node = pg_node.make_simple(
    base_dir=os.path.join(init_params.tmp_path, 'pg_node'),
    pg_options={
        "unix_socket_directories": "/tmp",
        "wal_level": "replica",
        "max_wal_senders": "2"
    }
)
node.status()

<NodeStatus.Stopped: 1>

b. Configure pg_hba.conf for replication connections
> echo "local replication all trust" >> pg_hba.conf

In [4]:
# Replication connections for streaming WAL
node.append_conf('pg_hba.conf', 'local replication all trust')
node.append_conf('pg_hba.conf', 'host replication all 127.0.0.1/32 trust')

# Regular connections for backup_user (needed for pg_backup_start function)
node.append_conf('pg_hba.conf', 'host all backup_user 127.0.0.1/32 trust')

PostgresNode(name='testgres-d7154e0c-1c51-4624-8444-8f404f3d4b16', port=14811, base_dir='/home/vshepard/PycharmProjects/backup_course/v7_full_backup_stream/pg_probackup_demo/pg_node')

c. Start the node
> pg_ctl start -D <data_directory>

In [5]:
node.slow_start()
print(f"PostgreSQL data directory: {node.data_dir}")
print(f"PostgreSQL port: {node.port}")
node.status()

PostgreSQL data directory: /home/vshepard/PycharmProjects/backup_course/v7_full_backup_stream/pg_probackup_demo/pg_node/data
PostgreSQL port: 14811


<NodeStatus.Running: 0>

d. Verify streaming replication settings
> psql -c "SHOW wal_level;"
> psql -c "SHOW max_wal_senders;"

In [16]:
wal_level = node.execute('postgres', 'SHOW wal_level;')[0][0]
max_wal_senders = node.execute('postgres', 'SHOW max_wal_senders;')[0][0]
print(f"wal_level: {wal_level}")
print(f"max_wal_senders: {max_wal_senders}")

wal_level: replica
max_wal_senders: 2


### 4. Create sample database with data

In [17]:
node.execute('postgres', 'DROP DATABASE IF EXISTS sales;')
node.execute('postgres', 'CREATE DATABASE sales;')

node.execute('sales', '''
CREATE TABLE products (
    id SERIAL PRIMARY KEY,
    name TEXT NOT NULL,
    price DECIMAL(10,2),
    created_at TIMESTAMP DEFAULT NOW()
);

INSERT INTO products (name, price) VALUES
    ('Laptop', 999.99),
    ('Mouse', 29.99),
    ('Keyboard', 79.99),
    ('Monitor', 299.99),
    ('Headphones', 149.99);

INSERT INTO products (name, price) VALUES
    ('Wrong_value', -100),
    ('Wrong_value 2', -98);

DELETE FROM products
WHERE price < 0;
''')

result = node.execute('sales', 'SELECT COUNT(*) FROM products;')
print(f"Created {result[0][0]} products in sales database")

Created 5 products in sales database


### 4a. Exploring WAL contents

Let's look at the pg_wal directory and see the actual WAL segments.
WAL segments are 16 MB files that contain all database changes.

In [18]:
import os

# Show pg_wal directory path
wal_dir = os.path.join(node.data_dir, 'pg_wal')
print(f"WAL directory: {wal_dir}\n")

# List WAL segment files (excluding archive_status directory)
wal_files = [f for f in os.listdir(wal_dir) if os.path.isfile(os.path.join(wal_dir, f))]
wal_files.sort()

print("WAL segment files:")
for f in wal_files:
    file_path = os.path.join(wal_dir, f)
    size = os.path.getsize(file_path)
    print(f"  {f}  ({size / 1024 / 1024:.1f} MB)")

WAL directory: /home/vshepard/PycharmProjects/backup_course/v7_full_backup_stream/pg_probackup_demo/pg_node/data/pg_wal

WAL segment files:
  000000010000000000000001  (16.0 MB)
  000000010000000000000002  (16.0 MB)


### 4b. Decoding WAL with pg_waldump

We can use `pg_waldump` utility to decode WAL contents and see the actual database changes.

**Useful pg_waldump options:**
- `-n 10` - show only first 10 records
- `-r heap` - filter for table operations (INSERT, UPDATE, DELETE)
- `-r Transaction` - filter for transaction records (COMMIT, ABORT)

> pg_waldump -r heap -n 15 000000010000000000000001

In [19]:
import subprocess

# Get pg_waldump path
pg_waldump = os.path.join(PG_BIN, 'pg_waldump')

# Get the first WAL segment file
if wal_files:
    first_wal = wal_files[0]
    wal_path = os.path.join(wal_dir, first_wal)
    
    print(f"Decoding WAL file: {first_wal}")
    print("Filtering for Heap records (table operations: INSERT, UPDATE, DELETE)\n")
    print("=" * 80)
    
    # Run pg_waldump with -r heap to filter for table operations
    result = subprocess.run(
        [pg_waldump, '-r', 'heap', '-n', '15', wal_path],
        capture_output=True,
        text=True
    )
    
    if result.returncode == 0 and result.stdout:
        print(result.stdout)
    else:
        # If no heap records in first file, try showing all records with skip
        print("No heap records in this segment yet.")
        print("\nShowing first 10 general records:")
        result = subprocess.run(
            [pg_waldump, '-n', '10', wal_path],
            capture_output=True,
            text=True
        )
        print(result.stdout if result.stdout else result.stderr)
else:
    print("No WAL files found")

Decoding WAL file: 000000010000000000000001
Filtering for Heap records (table operations: INSERT, UPDATE, DELETE)

rmgr: Heap        len (rec/tot):     59/  8183, tx:          3, lsn: 0/010C8160, prev 0/010C8138, desc: LOCK xmax: 3, off: 34, infobits: [LOCK_ONLY, EXCL_LOCK], flags: 0x00, blkref #0: rel 1663/1/1259 blk 0 FPW
rmgr: Heap        len (rec/tot):     73/  6617, tx:          3, lsn: 0/010CA170, prev 0/010C8160, desc: UPDATE old_xmax: 3, old_off: 34, old_infobits: [], flags: 0x00, new_xmax: 0, new_off: 36, blkref #0: rel 1663/1/1259 blk 5 FPW, blkref #1: rel 1663/1/1259 blk 0
rmgr: Heap        len (rec/tot):    189/   189, tx:          4, lsn: 0/010CFFA8, prev 0/010CFF88, desc: INSERT+INIT off: 1, flags: 0x00, blkref #0: rel 1663/1/2606 blk 0
rmgr: Heap        len (rec/tot):     59/  8175, tx:          4, lsn: 0/010D03C8, prev 0/010D0388, desc: LOCK xmax: 4, off: 37, infobits: [LOCK_ONLY, EXCL_LOCK], flags: 0x00, blkref #0: rel 1663/1/2610 blk 0 FPW
rmgr: Heap        len (rec/t

In [20]:
# Also show Transaction records (COMMIT operations)
print("Transaction records (COMMIT):")
print("=" * 80)

result = subprocess.run(
    [pg_waldump, '-r', 'Transaction', '-n', '5', wal_path],
    capture_output=True,
    text=True
)

if result.returncode == 0 and result.stdout:
    print(result.stdout)
else:
    print("No transaction records found or error:", result.stderr)

Transaction records (COMMIT):
rmgr: Transaction len (rec/tot):     98/    98, tx:          3, lsn: 0/010CFEB8, prev 0/010CED70, desc: COMMIT 2026-02-16 15:45:55.944891 CET; relcache init file inval dbid 1 tsid 1663; inval msgs: catcache 55 catcache 54 relcache 1260
rmgr: Transaction len (rec/tot):    114/   114, tx:          4, lsn: 0/010D4F00, prev 0/010D4208, desc: COMMIT 2026-02-16 15:45:55.945916 CET; relcache init file inval dbid 1 tsid 1663; inval msgs: catcache 19 catcache 32 relcache 2690 relcache 1255
rmgr: Transaction len (rec/tot):     66/    66, tx:          5, lsn: 0/010D51F0, prev 0/010D51B0, desc: COMMIT 2026-02-16 15:45:55.946186 CET; inval msgs: catcache 19
rmgr: Transaction len (rec/tot):    114/   114, tx:          6, lsn: 0/010D5518, prev 0/010D54C8, desc: COMMIT 2026-02-16 15:45:55.946314 CET; relcache init file inval dbid 1 tsid 1663; inval msgs: catcache 19 catcache 32 relcache 2703 relcache 1247
rmgr: Transaction len (rec/tot):     66/    66, tx:          7, lsn

### 4c. Understanding WAL record types

**Common WAL record types:**

| Type | Description |
|------|-------------|
| `Heap/INSERT` | Row inserted into a table |
| `Heap/UPDATE` | Row updated |
| `Heap/DELETE` | Row deleted |
| `Transaction/COMMIT` | Transaction committed |
| `XLOG/CHECKPOINT_ONLINE` | Checkpoint marker |
| `Btree/INSERT_LEAF` | Index entry added |

Each record shows:
- **LSN (Log Sequence Number)** - unique position in WAL
- **Resource manager** - which component created the record
- **Record type** - specific operation
- **Block references** - affected database pages

### 5. Initialize backup catalog and add instance

> pg_probackup init -B /mnt/backups

In [21]:
pb.init()

"INFO: Backup catalog '/home/vshepard/PycharmProjects/backup_course/v7_full_backup_stream/pg_probackup_demo/rel_backup_dir/backup_demo' successfully initialized\n"

> pg_probackup add-instance -B /mnt/backups -D /var/lib/pgpro/std-17/data --instance=main

In [22]:
pb.add_instance(instance='main', node=node)

"INFO: Instance 'main' successfully initialized\n"

### 6. Theory: FULL backup command parameters

Command to create a full backup in STREAM mode:

```bash
pg_probackup backup \
    -B /backups \
    --instance main \
    -b full \
    --stream
```

**Required parameters:**

| Parameter | Description |
|-----------|-------------|
| `backup` | Subcommand for creating a backup |
| `-B /backups` | Path to the backup repository |
| `--instance main` | Name of the PostgreSQL instance |
| `-b full` | Backup type (FULL, DELTA, PAGE, PTRACK) |
| `--stream` | WAL via streaming protocol |

**Why must the first backup be FULL?**

Incremental backups (DELTA, PAGE, PTRACK) are built on top of a full backup. Without a FULL backup, increments are impossible.

### 7. Create backup user with minimal privileges

In production, you should not use superuser for backups. Let's create a dedicated backup user with minimal privileges and see what happens.

> CREATE USER backup_user WITH PASSWORD 'secret';

In [23]:
node.execute('postgres', "CREATE USER backup_user WITH PASSWORD 'secret';")
print("Created user 'backup_user' with minimal privileges")

Created user 'backup_user' with minimal privileges


### 8. Attempt backup with insufficient privileges (will fail)


Let's try to create a backup using the new user without any special privileges:

 `ERROR:  permission denied for function pg_backup_start`

> pg_probackup backup -B /mnt/backups --instance=main -b FULL --stream --pguser=backup_user

In [24]:
from pg_probackup2.app import ProbackupException

try:
    pb.backup_node(
        instance='main',
        node=node,
        backup_type='full',
        options=['--stream', '--pguser=backup_user']
    )
except ProbackupException as e:
    print(e)


 ERROR: 'INFO: Backup start, pg_probackup version: 2.5.16, instance: main, backup ID: TAK45X, backup mode: FULL, wal mode: STREAM, remote: false, compress-algorithm: none, compress-level: 1\nINFO: This PostgreSQL instance was initialized with data block checksums. Data block corruption will be detected\nINFO: Database backup start\nINFO: wait for pg_backup_start()\nERROR: query failed: ERROR:  permission denied for function pg_backup_start\nquery was: SELECT pg_catalog.pg_backup_start($1, $2)\nWARNING: Backup TAK45X is running, setting its status to ERROR\n'
 CMD: /usr/local/pgsql-17/bin/pg_probackup backup -B /home/vshepard/PycharmProjects/backup_course/v7_full_backup_stream/pg_probackup_demo/rel_backup_dir/backup_demo --instance=main -p 14811 -d postgres -b full --no-sync --stream --pguser=backup_user


### 9. Grant required privileges for backup

For pg_probackup to work, the backup user needs:

1. **REPLICATION** privilege - to use streaming replication protocol for WAL
2. **EXECUTE** on specific pg_catalog functions - for backup operations

Reference: [pg_probackup documentation](https://postgrespro.com/docs/postgrespro/current/app-pgprobackup#PBK-CONFIGURING-THE-DATABASE-CLUSTER)

In [25]:
# Grant REPLICATION privilege (required for STREAM mode)
node.execute('postgres', 'ALTER USER backup_user WITH REPLICATION;')

# Grant EXECUTE on required pg_catalog functions
grants = [
    'GRANT EXECUTE ON FUNCTION pg_catalog.current_setting(text) TO backup_user',
    'GRANT EXECUTE ON FUNCTION pg_catalog.set_config(text, text, boolean) TO backup_user',
    'GRANT EXECUTE ON FUNCTION pg_catalog.pg_is_in_recovery() TO backup_user',
    'GRANT EXECUTE ON FUNCTION pg_catalog.pg_backup_start(text, boolean) TO backup_user',
    'GRANT EXECUTE ON FUNCTION pg_catalog.pg_backup_stop(boolean) TO backup_user',
    'GRANT EXECUTE ON FUNCTION pg_catalog.pg_switch_wal() TO backup_user',
    'GRANT EXECUTE ON FUNCTION pg_catalog.pg_create_restore_point(text) TO backup_user',
    'GRANT EXECUTE ON FUNCTION pg_catalog.pg_last_wal_replay_lsn() TO backup_user',
    'GRANT EXECUTE ON FUNCTION pg_catalog.txid_current() TO backup_user',
    'GRANT EXECUTE ON FUNCTION pg_catalog.txid_current_snapshot() TO backup_user',
    'GRANT EXECUTE ON FUNCTION pg_catalog.txid_snapshot_xmax(txid_snapshot) TO backup_user',
    'GRANT EXECUTE ON FUNCTION pg_catalog.pg_control_checkpoint() TO backup_user',
]

for grant in grants:
    node.execute('postgres', grant)
    func_name = grant.split('FUNCTION ')[1].split(' TO')[0]
print("All required privileges granted!")

All required privileges granted!


### 10. Verify user privileges

Let's check what privileges our backup_user has now:

In [26]:
result = node.execute('postgres', '''
    SELECT 
        rolname,
        rolsuper,
        rolreplication,
        ARRAY(
            SELECT b.rolname
            FROM pg_catalog.pg_auth_members m
            JOIN pg_catalog.pg_roles b ON (m.roleid = b.oid)
            WHERE m.member = r.oid
        ) as member_of
    FROM pg_roles r
    WHERE rolname = 'backup_user';
''')
print(f"User: {result[0][0]}")
print(f"Superuser: {result[0][1]}")
print(f"Replication: {result[0][2]}")
print(f"Member of roles: {result[0][3]}")

User: backup_user
Superuser: False
Replication: True
Member of roles: []


### 11. Create FULL backup with STREAM mode (success)

Now let's try the backup again with the properly configured user:

> pg_probackup backup -B /mnt/backups --instance=main -b FULL --stream --pguser=backup_user

In [27]:
backup_id = pb.backup_node(
    instance='main',
    node=node,
    backup_type='full',
    options=['--stream', '--pguser=backup_user']
)
print(f"Backup ID: {backup_id}")

Backup ID: TAK47B


### 12. View backup information

> pg_probackup show -B /mnt/backups --instance=main

**What you can see in the output:**
- `Mode: FULL` - backup type
- `WAL Mode: STREAM` - WAL stored inside the backup
- `Status: OK` - backup was successfully created and validated
- `Data` - data size
- `WAL` - WAL files size

In [28]:
print(pb.show(as_text=True, as_json=False))


BACKUP INSTANCE 'main'
 Instance  Version  ID      Recovery Time           Mode  WAL Mode  TLI  Time  Data   WAL  Zratio  Start LSN  Stop LSN   Status 
 main      17       TAK47B  2026-02-16 15:57:12+01  FULL  STREAM    1/0   10s  29MB  16MB    1,00  0/3000028  0/300B420  OK     
 main      17       TAK45X  ----                    FULL  STREAM    0/0     0     0     0    1,00  0/0        0/0        ERROR  



### 13. Backup catalog structure

Backup directory structure:

```
/backups/main/backups/<backup_id>/
├── database/              # Copy of PGDATA
│   ├── base/
│   ├── global/
│   ├── pg_wal/            # WAL files (in STREAM mode)
│   │   └── 000000010000000000000002
│   └── ...
├── backup.control         # Backup metadata
├── backup_content.control # File list with checksums
└── page_header_map        # Page header information
```

In STREAM mode, WAL files are stored inside `database/pg_wal/` - the same location as in a regular PostgreSQL data directory. This makes the backup **self-contained** and independent from external WAL archive.

### 14. Validate backup

> pg_probackup validate -B /mnt/backups --instance=main

**What validation checks:**
- Integrity of all files (checksums)
- Presence of all required WAL segments
- Correctness of backup structure

If validation fails, the backup gets status `ERROR` or `CORRUPT`.

In [29]:
print(pb.validate(backup_id=backup_id, instance='main'))

INFO: Validating backup TAK47B
INFO: Backup TAK47B data files are valid
INFO: Backup TAK47B WAL segments are valid
INFO: Backup TAK47B is valid.
INFO: Validate of backup TAK47B completed.



### 15. Create compressed backup

To save space, you can use compression:

```bash
pg_probackup backup -B /mnt/backups --instance=main -b FULL --stream \
    --compress-algorithm=zlib --compress-level=6
```

**Compression algorithms:**
- `none` - no compression (default)
- `zlib` - good compression, medium speed
- `lz4` - fast compression, lower ratio
- `zstd` - balance of speed and compression

In [30]:
# Add more data before second backup
node.execute('sales', '''
INSERT INTO products (name, price) VALUES
    ('Tablet', 449.99),
    ('Webcam', 89.99),
    ('USB Hub', 39.99);
''')
print("Added 3 more products")

Added 3 more products


In [31]:
backup_id_compressed = pb.backup_node(
    instance='main',
    node=node,
    backup_type='full',
    options=[
        '--stream',
        '--pguser=backup_user',
        '--compress-algorithm=zlib',
        '--compress-level=6'
    ]
)
print(f"Compressed Backup ID: {backup_id_compressed}")

Compressed Backup ID: TAK4BT


### 16. Compare backups

Let's compare backup sizes - with and without compression:

In [32]:
print(pb.show(instance='main', as_text=True, as_json=False))

 Instance  Version  ID      Recovery Time           Mode  WAL Mode  TLI  Time    Data   WAL  Zratio  Start LSN  Stop LSN   Status 
 main      17       TAK4BT  2026-02-16 15:59:55+01  FULL  STREAM    1/0   10s  9775kB  16MB    3,08  0/5000028  0/5000188  OK     
 main      17       TAK47B  2026-02-16 15:57:12+01  FULL  STREAM    1/0   10s    29MB  16MB    1,00  0/3000028  0/300B420  OK     
 main      17       TAK45X  ----                    FULL  STREAM    0/0     0       0     0    1,00  0/0        0/0        ERROR  



### Cleanup

In [33]:
node.stop()

import shutil
shutil.rmtree(os.environ['PGPROBACKUP_TMP_DIR'])